In [9]:
%run "../../../setup.ipynb"
if CLIENT == None:
    print("Failed to create CLIENT.")

Setup Complete. Use CLIENT to access Weaviate.


In [10]:
%pip install langchain
%pip install openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
import os
import openai

# openai.api_key = os.getenv("OPENAI_API_KEY")
deployment = os.getenv("AZURE_OPENAI_LLM_DEPLOYMENT_ID")
resource = os.getenv("AZURE_OPENAI_RESOURCE_NAME")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

In [16]:
from langchain.chat_models import AzureChatOpenAI

chat = AzureChatOpenAI(deployment_name=deployment)

In [24]:
from langchain.vectorstores import Weaviate

import weaviate

index_name = "Work"
text_key = "title"

weaviate = Weaviate(CLIENT, index_name, text_key)

In [37]:
result = weaviate.similarity_search_by_text(query="football", additional="certainty")
print(result)

[Document(page_content='Lingyin Monastery. Feilaifeng. Exterior. Detail. Sculptural roundel', metadata={'_additional': {'certainty': 0.6787058413028717}}), Document(page_content='Africa', metadata={'_additional': {'certainty': 0.6763530075550079}}), Document(page_content='Wuta Si (Five-Pagoda Temple) Zhenjue Monastery. Exterior. Stūpa. Detail', metadata={'_additional': {'certainty': 0.6760348081588745}}), Document(page_content='Receipt of W.H. Shaffer from Benjamin Auld. Benjamin F. Auld to W.H. Shaffer. Baltimore, Maryland. 1p. Receipt of sale for Frederick Douglass materials.', metadata={'_additional': {'certainty': 0.6755401492118835}})]
